In [1]:
import sys
sys.path.append("../../")
import json
import urllib.request
from IPython.display import clear_output
import sources
from countries.list import CountryList
from data.Frame import Frame
from data.Stats import calculate_weighted_average as wa
c = CountryList()
c.load_wb()
from sources.worldbank.indicators import get_data_frame_wide as getdfw
from sources.worldbank.datatable import get_data_table

In [14]:
years = ["2001", "2011", "2016", "2017", "2018"]
groups_ldcs=[['LDCs']]
indicator = 'NV.AGR.TOTL.ZS'
weight = 'NY.GDP.MKTP.CD'

In [15]:
def estimate(years,indicator, groups, weight = None):
    data_df = get_data_table(years, indicator, groups, export_to_excel=False)    
    
    if weight is None:
        return (data_df)    
    
    weight_df = get_data_table(years, weight, groups, export_to_excel=False)   
    
    df_ldcs_w = wa(data_df, weight_df, 'entity' )
    
    return (df_ldcs_w)

In [16]:
data = estimate(years, indicator, groups_ldcs, weight)

In [17]:
data.get_last_row().get_column_names()

['entity',
 '2001.0_value',
 '2001.0_weight',
 '2001.0_weighted_value',
 '2011.0_value',
 '2011.0_weight',
 '2011.0_weighted_value',
 '2016.0_value',
 '2016.0_weight',
 '2016.0_weighted_value',
 '2017.0_value',
 '2017.0_weight',
 '2017.0_weighted_value',
 '2018.0_value',
 '2018.0_weight',
 '2018.0_weighted_value']

In [20]:
data.get_last_row()[0]

Average

In [21]:
data.get_last_row()[3]

27.21827502955226

In [22]:
data.get_last_row()[6]

21.011183599950137

In [23]:
data.get_last_row()[9]

21.26276969611124

In [29]:
data.get_last_row()[12]

20.813614631808505

In [30]:
data.get_last_row()[15]

19.9670482288127